In [ ]:
%pip install pymysql
%pip install sshtunnel
%pip install pandas

In [3]:
import pymysql
from sshtunnel import SSHTunnelForwarder
import paramiko
from os.path import expanduser
import pandas as pd

In [4]:
import numpy as np

In [2]:
import tensorflow as tf
print(tf.__version__)

Init Plugin
2.5.0
Init Graph Optimizer
Init Kernel


In [5]:
home = expanduser('~')
mypkey = paramiko.RSAKey.from_private_key_file( home + "/.ssh/id_rsa")

sql_hostname = '127.0.0.1'
sql_username = 'readonly'
sql_password = None
sql_main_database = 'ML3_mirror'
sql_port = 3306
ssh_host = 'flagon.cs.umn.edu'
ssh_user = 'kanna128'
ssh_port = 22
sql_ip = '1.1.1.1.1'

def table1_data():
  with SSHTunnelForwarder(
          (ssh_host, ssh_port),
          ssh_username=ssh_user,
          ssh_pkey=mypkey,
          remote_bind_address=(sql_hostname, sql_port)) as tunnel:
      conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                              passwd=sql_password, db=sql_main_database,
                              port=tunnel.local_bind_port)
      print('connection done')

      # query to calculate the number of users
      query_users = ''' SELECT expt_user.group, expt_user.userId from ML3_mirror.expt_user WHERE 
                        exptId = 'listcmp-long' AND started <'2015-04-01' 
                        AND status = 'RANDOMLY_ASSIGNED' '''
      users = pd.read_sql_query(query_users, conn)
      users_count = users.count()['group']
      print("Users: {}".format(users_count))

      recchange_dict = {} # dict to store key=userId, val=curralg in the iteration process
      initial_alg = {}

      # key=userId; val=changes list of tuples with prev, next algorithms
      changes_dict = {} # needed to later calculate median changes per user
      for k in range(users_count):
        userId = users.at[k, 'userId']
        group = "{\"engineId\":\""+users.at[k, 'group']+"\"}" # following the format of log_action.log_Json for future comparisons
        recchange_dict[userId] = group
        initial_alg[userId] = group
        changes_dict[userId] = 0
      
      # m=0
      # for k,v in recchange_dict.items():
      #   print(k, v)
      #   m = m + 1
      #   if m==10:
      #     break

      # query to calculate the number of recommender change events
      # note: adding/ removing started<'2015-04-01' inside the EXISTS(...) part doesn't change the count
      # avoid variable separation
      # ORDER BY: to sort by timestamp

      # note: ORDER BY tstamp doesn't change results as of now
      query_change = '''SELECT userId, logJson, tstamp, action from ML3_mirror.log_action WHERE EXISTS 
                        (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
                        AND expt_user.exptId='listcmp-long' 
                        AND expt_user.status='RANDOMLY_ASSIGNED') 
                        AND log_action.action='recommender-change' 
                        AND log_action.tstamp<'2015-04-01' ORDER BY log_action.tstamp'''
      change_events = pd.read_sql_query(query_change, conn)
      print("query done")
      # change_events.to_csv("df2")
      change_events.sort_values(by=['userId'])
      # print(change_events)

      length = change_events.count()['userId']
      # print(length)
      
      changes = 0
      for i in range(length):
        # print(i)
        userId = change_events.at[i, 'userId']
        recalg = change_events.at[i, 'logJson']
        if(recchange_dict[userId] != recalg):
          changes += 1
          changes_dict[userId] += 1
        
        recchange_dict[userId] = recalg
      # print(userId, recalg, "hello")
      
      
      keys = changes_dict.keys()
      # for val in changes_dict.values():
      #   if(val != 0):
      #     at_least_once += 1
      
      changes_dict = {key:val for key,val in changes_dict.items() if val !=0}
      at_least_once = len(changes_dict.values())
      

      print("Fig3_basic start")
      bchange = 0
      ichange = 0
      pchange = 0
      schange = 0

      bas = 0
      item = 0
      pick = 0
      s = 0

      # print(type(initial_alg))
      # k = 0
      # for key in initial_alg.keys():
      #   if(k<5):
      #     print(key)
      #     print(initial_alg[key])
      #   k = k + 1

      for userId,recalg in initial_alg.items():
        if(recalg == "{\"engineId\":\"baseline\"}"):
          bas += 1
        if(recalg == "{\"engineId\":\"item-item\"}"):
          item += 1
        if(recalg == "{\"engineId\":\"pick-groups\"}"):
          pick += 1
        if(recalg == "{\"engineId\":\"svd\"}"):
          s += 1

        if userId in changes_dict.keys():
          if(recalg == "{\"engineId\":\"baseline\"}"):
            bchange += 1
          if(recalg == "{\"engineId\":\"item-item\"}"):
            ichange += 1
          if(recalg == "{\"engineId\":\"pick-groups\"}"):
            pchange += 1
          if(recalg == "{\"engineId\":\"svd\"}"):
            schange += 1

      bchange = bchange/bas
      ichange = ichange/item
      schange = schange/s

      print("bchange: ", bchange)
      print("ichange: ", ichange)
      print("pchange: ", pchange)
      print("schange: ", schange)

      print("Fig3_basic done")

      baseline = 0
      item_item = 0
      pick_groups = 0
      svd = 0

      # repeated conditional statements -- are they okay?
      # alternative: dictionary --> key=string to compare, val=variable to increment
      # tradeoff: additional memory usage 
      # the tradeoff probably doesn't matter much
      # an immediate access mechanism to increment variables is better than a series of if statements holding the loop back for a while 
      # time this part of the program to experiment with these ideas
      for userId, recalg in recchange_dict.items():
        if(userId in changes_dict.keys()):
          if(recalg == "{\"engineId\":\"baseline\"}"): # format of string to compare with is the same as format of log_action.logJson
            baseline += 1
          if(recalg == "{\"engineId\":\"item-item\"}"):
            item_item += 1
          if(recalg == "{\"engineId\":\"pick-groups\"}"):
            pick_groups += 1
          if(recalg == "{\"engineId\":\"svd\"}"):
            svd += 1

      print("Recommender change events: ", changes)
      print("Users switching at least once: ", at_least_once)
      # print(type(list(changes_dict.values())))
      print("Median changes per user w/ at least 1 change: ", int(np.median(list(changes_dict.values()))))
      print("Baseline: ", baseline)
      print("Item-item: ", item_item)
      print("Pick Groups: ", pick_groups)
      print("Svd: ", svd)

      # expt_end_date - account_creation_date as a count of days

if __name__ == "__main__":
  table1_data()
  print('done')

connection done
Users: 3005
query done
Fig3_basic start
bchange:  0.296875
ichange:  0.2469879518072289
pchange:  0
schange:  0.20101522842639594
Fig3_basic done
Recommender change events:  11418
Users switching at least once:  748
Median changes per user w/ at least 1 change:  3
Baseline:  54
Item-item:  290
Pick Groups:  62
Svd:  342
done


,Unnamed: 0,userId,logJson,tstamp,action
0,0,84090,"{""engineId"":""pick-groups""}",2014-10-31 15:22:45,recommender-change
1,1,84090,"{""engineId"":""baseline""}",2014-10-31 15:23:13,recommender-change
2,2,84090,"{""engineId"":""pick-groups""}",2014-10-31 15:23:37,recommender-change
3,3,84090,"{""engineId"":""baseline""}",2014-10-31 15:23:46,recommender-change
4,4,84090,"{""engineId"":""pick-groups""}",2014-10-31 15:23:53,recommender-change
